In [1]:
# Some steps required until these Python modules are properly installed...
import sys
sys.path.append('../../Modules')
sys.path.append('../../../../veneer-py')
# Get the Source scripting components (veneer) and GBR specific stuff
import gbr
import fiona
%pylab inline
import pandas as pd
import geopandas as gpd
#
# RC7 current load, hillsope load
#
gbr.init('D:/a_sc_p2r_b_rc7/Model_runs/BD_1/Runs/a_scen_reader/RC7/')
gbr.available()
    #
results = gbr.Results('CC')
CC_raw = results.get('RawResults')
    # Filter raw results for catchment data (ie not link related)
catchmentData = CC_raw[CC_raw.ModelElementType=='Catchment']
constituent = 'Sediment - Fine'
    # Filter again for a constituent of interest and a process...
    # TODO - should be able to do all that follows for all constituents/processes and generate a single output shapefile/dbf
constituent = 'Sediment - Fine'
process = 'Hillslope surface soil'
relevantData = catchmentData[np.logical_and(catchmentData.index==constituent,catchmentData.BudgetElement==process)].copy()
    # Convert load to tons/year
    # TODO: tons/ha/year - could load the FU areas table from the results, OR rely on the polygon areas
conversion = 1000.0*results.runDetails.yearsOfRecording
relevantData['Load_t_y'] = relevantData['Total_Load_in_Kg']/conversion
del relevantData['Total_Load_in_Kg']
    # Reindex the data by subcatchment and FU, drop other columns (all the same value anyway)
CL_loadData = relevantData[['ModelElement','FU','Load_t_y']]
CL_loadData = CL_loadData.set_index(['ModelElement','FU'])
#
# RC7 All Change, hillsope load
#
gbr.init('D:/a_sc_p2r_b_rc7/Model_runs/BD_1/Runs/a_scen_reader/RC7/')
gbr.available()
    #
results = gbr.Results('all_change')
all_change_raw = results.get('RawResults')
    #
    # Filter raw results for catchment data (ie not link related)
catchmentData = all_change_raw[all_change_raw.ModelElementType=='Catchment']
constituent = 'Sediment - Fine'
    # Filter again for a constituent of interest and a process...
    # TODO - should be able to do all that follows for all constituents/processes and generate a single output shapefile/dbf
constituent = 'Sediment - Fine'
process = 'Hillslope surface soil'
relevantData = catchmentData[np.logical_and(catchmentData.index==constituent,catchmentData.BudgetElement==process)].copy()
    # Convert load to tons/year
    # TODO: tons/ha/year - could load the FU areas table from the results, OR rely on the polygon areas
conversion = 1000.0*results.runDetails.yearsOfRecording
relevantData['Load_t_y'] = relevantData['Total_Load_in_Kg']/conversion
del relevantData['Total_Load_in_Kg']
    # Reindex the data by subcatchment and FU, drop other columns (all the same value anyway)
all_change_loadData = relevantData[['ModelElement','FU','Load_t_y']]
all_change_loadData = all_change_loadData.set_index(['ModelElement','FU'])
#
# RC7 CL_loadData - all_change_loadData
#
CC_all_Change_Diff = CL_loadData-all_change_loadData
#
# Read in intsFu shapefile
#
b_intsFu = gpd.read_file('D:/a_sc_p2r_b_rc7/Model_runs/BD_1/Configuration/bu_intSCFU.shp')
b_intsFu.rename(columns={'reef_luse':'FU','IntSCFU':'DROP','Column_0':'ModelElement'},inplace=True)
b_intsFu.set_index(['ModelElement','FU'],inplace=True)
#
# Join with CC_all_Change_Diff
#
spatialLoads = b_intsFu.join(CC_all_Change_Diff)
spatialLoads.geometry = gpd.GeoSeries(spatialLoads.geometry)
#
# Write shapefile
#
spatialLoads.to_file('D:/a_sc_p2r_b_rc7/Model_runs/BD_1/Runs/a_scen_reader/RC7/diff_CL_all_Change_hillsope_fs_.shp')

Populating the interactive namespace from numpy and matplotlib
